In [1]:
import pandas as pd
import numpy as np

train_data = pd.read_excel("Data/tr.xlsx",)

In [2]:
over_missing_sample = []
row_value_count = train_data.apply(pd.Series.value_counts,axis=1,dropna=False)

In [38]:
import tools
#Data transform
train_y = pd.DataFrame(train_data['outcome']) 
train_X = pd.DataFrame(train_data.drop(['outcome'],axis=1))

train_X = tools.data_transform(train_X)

In [39]:
threshhold =len(train_X.columns)*0.25
over_missing = row_value_count[np.nan]<=threshhold
class_0 = train_y['outcome']==1

print(sorted(train_y.value_counts()))
train_X = train_X[over_missing|class_0]
train_y = train_y[over_missing|class_0]
print(sorted(train_y.value_counts()))


[879, 51280]
[879, 27076]


In [40]:
# Data split
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(
    train_X ,
    train_y,
    test_size=0.2,
    random_state=42)

print(sorted(train_y.value_counts()))
print(sorted(val_y.value_counts()))

[718, 21646]
[161, 5430]


In [41]:
from imblearn.under_sampling import RandomUnderSampler as RUS

print(sorted(train_y.value_counts()))

rus = RUS(sampling_strategy=0.1,random_state=42)
train_X,train_y = rus.fit_resample(train_X,train_y)

print(sorted(train_y.value_counts()))

[718, 21646]
[718, 7180]


In [42]:
from sklearn.feature_selection import VarianceThreshold as VT

vt= VT(0.2)

vt.fit(train_X)
print(train_X.shape)

train_X = pd.DataFrame(vt.transform(train_X)) 
val_X =  pd.DataFrame(vt.transform(val_X))

print(train_X.shape)

(7898, 66)
(7898, 30)


In [43]:
# Missing value imputation
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import tools
feature_kind = tools.init_feature_kind(train_X)
cont,cate = tools.get_feature_kind(train_X,feature_kind)  

strategy = 'median'

imp_mean = IterativeImputer(max_iter=50,random_state=0)
imp_mean.fit(train_X[cont])

train_X[cont] = imp_mean.transform(train_X[cont])
val_X[cont] = imp_mean.transform(val_X[cont])

print("filled continuous missing value with "+strategy)

strategy = 'constant'

imp = SimpleImputer(missing_values=np.nan, strategy=strategy,fill_value=10.0)
imp.fit(train_X[cate])

train_X[cate] = imp.transform(train_X[cate])
val_X[cate] = imp.transform(val_X[cate])


print("filled nominal missing value with ",strategy)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\impute\_iterative.py:685: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"


filled continuous missing value with median
filled nominal missing value with  constant


In [44]:
from sklearn import preprocessing


cont,cate = tools.get_feature_kind(train_X,feature_kind)

scaler = preprocessing.StandardScaler().fit(train_X[cont])
train_X[cont] = scaler.transform(train_X[cont])
val_X[cont] = scaler.transform(val_X[cont])

In [45]:
'''import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import RidgeCV

ridge = RidgeCV(alphas=np.logspace(-6, 6, num=5)).fit(train_X, train_y)
importance = np.abs(ridge.coef_)
feature_names = np.array(train_X.columns)
plt.bar(height=importance[0], x=feature_names)
plt.title("Feature importances via coefficients")
plt.show()


train_X=train_X[feature_names[importance[0]>0.004]]
val_X = val_X[feature_names[importance[0]>0.004]]
print(train_X.shape)'''

'import matplotlib.pyplot as plt\nimport numpy as np\nfrom sklearn.linear_model import RidgeCV\n\nridge = RidgeCV(alphas=np.logspace(-6, 6, num=5)).fit(train_X, train_y)\nimportance = np.abs(ridge.coef_)\nfeature_names = np.array(train_X.columns)\nplt.bar(height=importance[0], x=feature_names)\nplt.title("Feature importances via coefficients")\nplt.show()\n\n\ntrain_X=train_X[feature_names[importance[0]>0.004]]\nval_X = val_X[feature_names[importance[0]>0.004]]\nprint(train_X.shape)'

In [46]:

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
#用RFE,返回特徵選擇後的資料
#參數estimator裡放機器學習模型
#參數n_feature_to_select為要選擇的特徵個數
#clf = LogisticRegression(class_weight={1:1,0:0.1},solver='liblinear')

#record = tools.wrapper_approach(clf,train_X,train_y,val_X,val_y)

In [47]:
def set_pandas_display_options() -> None:
    display = pd.options.display
    display.max_columns = 100
    display.max_rows = 100
    display.max_colwidth = 199
    display.width = None
set_pandas_display_options()

In [48]:
'''import seaborn as sns
import matplotlib.pyplot as plt
#print(record)
df = pd.DataFrame(record).T
#print(df)
df.plot.line()
plt.title('wrapper approach on LR with PCA')
print(pd.DataFrame(record).T)'''

"import seaborn as sns\nimport matplotlib.pyplot as plt\n#print(record)\ndf = pd.DataFrame(record).T\n#print(df)\ndf.plot.line()\nplt.title('wrapper approach on LR with PCA')\nprint(pd.DataFrame(record).T)"

In [49]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
clf = LogisticRegression(class_weight={1:1,0:0.1},max_iter=10000) 

parameters = {
    'penalty':('l1', 'l2','elasticnet', 'none'),
    'C':[1, 10,0.1,100],
    'solver':('newton-cg','lbfgs', 'liblinear', 'sag', 'saga'),
    
    }

GS = GridSearchCV(clf, parameters,n_jobs=5,scoring='f1')
GS.fit(train_X, train_y['outcome'])


sorted(GS.cv_results_.keys())

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.2554991         nan 0.25694731 0.25537381
 0.25537381 0.25529685 0.25537381 0.25565193        nan        nan
        nan        nan        nan 0.25388267 0.25310557        nan
 0.25311259 0.25340645        nan        nan 0.25455777        nan
 0.25414437 0.2524689  0.25296575 0.25283566 0.25312649 0.25350891
        nan        nan        nan        nan        nan 0.25388267
 0.25310557        nan 0.25311259 0.25340645        nan        nan
 0.24405033        nan 0.25148202 0.25580647 0.2558853  0.2526244
 0.25580647 0.2557318         nan        nan        nan        nan
        nan 0.25388267 0.25310557        nan 0.25311259 0.25340645
        nan        nan 0.25333198        nan 0.25386078 0.25310557
 0.25318324 0.25310557 0.25325603 0.25348456        nan        nan
        nan        nan        nan 0.25388267 0.2531055

['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'param_C',
 'param_penalty',
 'param_solver',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split1_test_score',
 'split2_test_score',
 'split3_test_score',
 'split4_test_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score']

In [50]:
print(GS.best_params_)
print(pd.DataFrame(GS.cv_results_))

{'C': 1, 'penalty': 'l1', 'solver': 'saga'}
    mean_fit_time  std_fit_time  mean_score_time  std_score_time param_C  \
0        0.002802  3.991131e-04         0.000000        0.000000       1   
1        0.002601  4.911295e-04         0.000000        0.000000       1   
2        0.264265  9.931706e-02         0.003797        0.000742       1   
3        0.002602  4.881408e-04         0.000000        0.000000       1   
4        2.729284  5.714271e-01         0.003193        0.000405       1   
5        0.102451  8.878321e-03         0.003200        0.000751       1   
6        0.104424  1.363802e-02         0.003202        0.000746       1   
7        0.058416  3.143673e-03         0.003197        0.000403       1   
8        0.987027  1.925412e-01         0.003393        0.000490       1   
9        1.828431  4.874221e-01         0.004000        0.000631       1   
10       0.002488  7.387212e-04         0.000000        0.000000       1   
11       0.002000  8.714517e-07         0.00

In [52]:
from sklearn.feature_selection import RFE

best_parameters = {
    'C': 1, 'penalty': 'l1', 'solver': 'saga'
}

clf = LogisticRegression(**best_parameters,class_weight={1:1,0:0.1},max_iter=10000) 

clf.fit(train_X, train_y['outcome'])
result = clf.predict(val_X)


print(pd.DataFrame({
    'train':tools.get_performance(train_y,clf.predict(train_X)),
    'val':tools.get_performance(val_y,result)
    }
    ))




                                        train                        val
confusion matrix   [[4887, 2293], [266, 452]]  [[3671, 1759], [61, 100]]
acc                                  0.675994                   0.674477
precision                            0.164663                   0.053792
f1_score                             0.261045                    0.09901
recall                               0.629526                   0.621118
matthews_corrcoef                    0.187249                   0.105491
